In [ ]:
#Enfoque basado en modelo de SentiWordNet o SenticNet para español
#Añadiendo reglas heurísticas (negación en ventana de máximo 5 tokens, intensificadores y atenuadores)
#Compara y evalúa cíticamente el rendimiento del modelo sin y con reglas heurísticas (lexicón y lexicón + reglas)
#Para comparar y evaluar, usa un dataset de evaluación: InterTASS, Multilingual Amazon Review sentiment...

In [ ]:
#Análisis_del_sentimiento_y_la_connotación_Aprendizaje_basado_en_el_lexicón
#Apartado "Lexicones de polaridad" utiliza el lexicón de SentiWordNet y añade reglas heurísticas, pero es en inglés
#Cuidado porque en español se hace con Spacy en lugar de con NLTK
#SentiWordNet en español en el apartado "Análisis de sentimiento basado en aspecto", dentro de "Lexicones de polaridad"

Ahora vamos a usar SentiWordNet en español. Cargaremos el lexicón.

In [ ]:
import csv
import spacy

In [ ]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 53.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [48]:
# Cargar el modelo de spaCy para español
nlp_es = spacy.load("es_core_news_sm")

# Función para cargar el lexicón de SentiWordNet en español.
def load_spanish_sentiwordnet(filepath):
    """
    Carga el lexicón en español desde un archivo tab-delimitado con las columnas:
    pos, word_en, word_sp, positive, negative, objective, index, synset, meaning.
    Se utiliza la columna 'word_sp' (convertida a minúsculas) como clave.
    Devuelve un diccionario: {word_sp.lower(): (pos_tag, positive, negative, objective)}.
    """
    lexicon = {}
    with open(filepath, encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter="\t")
        for row in reader:
            word_sp = row["word_sp"].strip().replace('_', ' ')
            if word_sp == "":
                continue
            try:
                pos_tag = row["pos"].strip().lower()
                pos_score = float(row["positive"].replace("+", ""))
                neg_score = float(row["negative"].replace("-", ""))
                obj_score = float(row["objective"])
                synset_code = row["synset"]
            except Exception:
                pos_score, neg_score, obj_score = 0.0, 0.0, 0.0
                pos_tag = ""
                synset_code = ""
            key = word_sp.lower()
            lexicon[key] = (pos_tag, pos_score, neg_score, obj_score, synset_code)
    return lexicon

In [49]:
# Cargar el lexicón en español desde el mismo cuaderno
# La ruta en la que he colocado el archivo es "/content/sentiwordnet_es.tsv"
es_sentiwordnet = '/content/sentiwordnet_es.tsv'

In [50]:
# Cargar el lexicón (ajusta la ruta al archivo de SentiWordNet en español)
spanish_sentiwordnet = load_spanish_sentiwordnet(es_sentiwordnet)
spanish_sentiwordnet

{'barrer con': ('v', 0.125, 0.0, 0.875, '01109259'),
 'ganar': ('v', 0.375, 0.0, 0.625, '02290461'),
 'abalizar': ('v', 0.125, 0.0, 0.875, '01218512'),
 'ligarse': ('v', 0.125, 0.0, 0.875, '01356750'),
 'unirse': ('v', 0.125, 0.0, 0.875, '01356750'),
 'seguir': ('v', 0.375, 0.0, 0.625, '00118764'),
 'raptar': ('v', 0.0, 0.125, 0.875, '01471043'),
 'cautivar': ('v', 0.125, 0.0, 0.875, '01817314'),
 'deambular': ('v', 0.125, 0.0, 0.875, '01882081'),
 'platyhelminthes': ('n', 0.125, 0.0, 0.875, '01924590'),
 'anomiidae': ('n', 0.125, 0.0, 0.875, '01962223'),
 'pluvialis': ('n', 0.125, 0.0, 0.875, '02024353'),
 'género pluvialis': ('n', 0.125, 0.0, 0.875, '02024353'),
 'kogia': ('n', 0.125, 0.0, 0.875, '02067462'),
 'considerar': ('v', 0.25, 0.0, 0.75, '02166460'),
 'vanessa': ('n', 0.125, 0.0, 0.875, '02275921'),
 'nesokia': ('n', 0.125, 0.0, 0.875, '02334079'),
 'género nesokia': ('n', 0.125, 0.0, 0.875, '02334079'),
 'socorrer': ('v', 0.125, 0.0, 0.875, '02548710'),
 'diaforético': ('a'

In [51]:
# Función que incluye el diccionario para mapear las etiquetas de spaCy a las etiquetas esperadas en el lexicón.
# Asumimos que en el lexicón se utilizan 'n' (noun), 'v' (verb), 'a' (adjective) y 'r' (adverb).
def map_spacy_tag(spacy_tag):

    spacy_to_wn = {
        "NOUN": "n",
        "VERB": "v",
        "ADJ": "a",
        "ADV": "r"
    }

    return spacy_to_wn.get(spacy_tag, None)

Ahora vamos a probar el lexicón de SentiWordNet en español para calcular el sentimiento.

Hasta ahora hemos hecho un mapeo entre las etiquetas gramaticales ya incluídas en el lexicón de SentiWordNet a las etiquetas que usa WordNet, centrándonos únicamente en categorías léxicas (nombres, verbos, adjetivos, adverbios). Después, vamos a lematizar los tokens, esto es, quitar cualquier afijo para obtener la raíz de los mismos. Buscaremos el lema en WordNet y obtendremos el significado del primero, que es el más común. Por último, ofrecemos dos maneras de calcular el sentimiento:

1. Mediante la suma de la resta del sentimiento positivo y negativo de cada synset por palabra.
2. Mediante la media de la suma de los sentimientos positivos, negativos y objetivos de todas las palabras, escogiendo el valor más alto de los tres sentimientos.

En el primer caso, consideraremos la misma escala de valores que vimos en VADER:
si s es superior o igual a 0.05, positivo; si s es inferior o igual a -0.05, negativo; en el resto de los casos, neutro.

En el segundo, calcularemos la media de la suma de los scores positivos, negativos y objetivos y tomaremos como polaridad aquel de los tres que tenga un valor mayor.

Ahora vamos a crear una función para cada una de los métodos para calcular el sentimiento y vamos a aplicar tal función al dataset anterior, reutilizando para ello el código del que nos servimos anteriormente para cargar el dataset, visualizarlo, manipularlo, etc.

In [77]:
def compute_sentiment_sum(text):

  doc = nlp_es(text)
  tokenized_sent = [token.text for token in doc]
  print(tokenized_sent)

  pos_tagged_tokens = [(token.text, token.pos_) for token in doc]
  print(pos_tagged_tokens)

  pos_score = 0.0
  neg_score = 0.0
  sentiment_value = 0.0
  sentiment_label = ""

  for word_text, tag in pos_tagged_tokens:
    print(word_text)
    wn_tag = map_spacy_tag(tag)
    print(wn_tag)
    if wn_tag == None:
     continue

    word = next((token for token in doc if token.text == word_text), None)
    if word is None:
      continue

    lemma = word.lemma_
    print(lemma)
    if not lemma:
      continue

    # Verificar si la palabra lematizada está en el diccionario de SentiWordNet
    # y crear la lista de synsets
    if lemma in spanish_sentiwordnet:
        # Obtener los puntajes positivos, negativos y los códigos de los synsets
        pos_tag, pos_score_lemma, _, neg_score_lemma, synset = spanish_sentiwordnet[lemma]

        swn_synsets = []
        for key, value in spanish_sentiwordnet.items():
          # Comprobar si el synset de la palabra coincide con el synset del diccionario
          if synset == value[4]:  # value[4] es el synset
              swn_synsets.append(f"SentiSynset('{key}.{pos_tag}')")

        print(swn_synsets)
        if len(swn_synsets) == 0:
          continue

        print(f"Positive score for {lemma}: {pos_score_lemma}")
        print(f"Negative score for {lemma}: {neg_score_lemma}")

        pos_score += pos_score_lemma
        neg_score += neg_score_lemma

        # Calcular el valor del sentimiento
        sentiment_value += pos_score - neg_score
        print(f"Sentiment value: {sentiment_value}")

  # Clasificación final según el valor de sentimiento
  if sentiment_value >= 0.05:
      sentiment_label = "positive"
  elif sentiment_value <= -0.05:
      sentiment_label = "negative"
  else:
      sentiment_label = "neutral"

  return sentiment_label


In [78]:
text ="Hoy me he levantado alegre considerando que estoy tristemente enfermo"

In [79]:
valor_sentimiento = compute_sentiment_sum(text)
print(valor_sentimiento)

['Hoy', 'me', 'he', 'levantado', 'alegre', 'considerando', 'que', 'estoy', 'tristemente', 'enfermo']
[('Hoy', 'ADV'), ('me', 'PRON'), ('he', 'AUX'), ('levantado', 'VERB'), ('alegre', 'ADJ'), ('considerando', 'VERB'), ('que', 'SCONJ'), ('estoy', 'AUX'), ('tristemente', 'ADV'), ('enfermo', 'NOUN')]
Hoy
r
hoy
["SentiSynset('hoy.n')"]
Positive score for hoy: 0.125
Negative score for hoy: 0.875
Sentiment value: -0.75
me
None
he
None
levantado
v
levantar
["SentiSynset('levantar.v')"]
Positive score for levantar: 0.0
Negative score for levantar: 0.875
Sentiment value: -2.375
alegre
a
alegre
considerando
v
considerar
["SentiSynset('considerar.v')", "SentiSynset('estudiar.v')"]
Positive score for considerar: 0.25
Negative score for considerar: 0.75
Sentiment value: -4.5
que
None
estoy
None
tristemente
r
tristemente
["SentiSynset('tristemente.r')"]
Positive score for tristemente: 0.0
Negative score for tristemente: 0.375
Sentiment value: -7.0
enfermo
n
enfermo
negative


In [89]:
def compute_sentiment_mean(text):

  doc = nlp_es(text)
  tokenized_sent = [token.text for token in doc]
  print(tokenized_sent)

  pos_tagged_tokens = [(token.text, token.pos_) for token in doc]
  print(pos_tagged_tokens)

  pos_score = 0.0
  neg_score = 0.0
  obj_score = 0.0
  num_lemas_con_synset = 0
  sentiment_label = "neutral"

  for word_text, tag in pos_tagged_tokens:

    wn_tag = map_spacy_tag(tag)
    if wn_tag == None:
     continue

    word = next((token for token in doc if token.text == word_text), None)
    if word is None:
      continue

    lemma = word.lemma_
    if not lemma:
      continue

    # Initialize pos_score_lemma, neg_score_lemma, and obj_score_lemma before the if statement
    pos_score_lemma = 0.0
    neg_score_lemma = 0.0
    obj_score_lemma = 0.0

    if lemma in spanish_sentiwordnet:
      # Obtener los puntajes positivos, negativos y los códigos de los synsets
      pos_tag, pos_score_lemma, obj_score_lemma, neg_score_lemma, synset = spanish_sentiwordnet[lemma]

      swn_synsets = []
      for key, value in spanish_sentiwordnet.items():
        # Comprobar si el synset de la palabra coincide con el synset del diccionario
        if synset == value[4]:  # value[4] es el synset
            swn_synsets.append(f"SentiSynset('{key}.{pos_tag}')")

      if len(swn_synsets) == 0:
        continue

    pos_score += pos_score_lemma
    neg_score += neg_score_lemma
    obj_score += obj_score_lemma
    num_lemas_con_synset += 1

  if num_lemas_con_synset > 0:
    pos_score_mean = pos_score / num_lemas_con_synset
    neg_score_mean = neg_score / num_lemas_con_synset
    obj_score_mean = obj_score / num_lemas_con_synset
    max_value = max(pos_score_mean, neg_score_mean, obj_score_mean)

    if max_value == pos_score_mean:
      sentiment_label = "positive"
    elif max_value == neg_score_mean:
      sentiment_label = "negative"

  return sentiment_label

In [90]:
valor_sentimiento_mean = compute_sentiment_mean(text)
print(valor_sentimiento_mean)

['Hoy', 'me', 'he', 'levantado', 'alegre', 'considerando', 'que', 'estoy', 'tristemente', 'enfermo']
[('Hoy', 'ADV'), ('me', 'PRON'), ('he', 'AUX'), ('levantado', 'VERB'), ('alegre', 'ADJ'), ('considerando', 'VERB'), ('que', 'SCONJ'), ('estoy', 'AUX'), ('tristemente', 'ADV'), ('enfermo', 'NOUN')]
negative


In [91]:
#Definición de una función que da formato a los datos de la tabla y los prepara para poder trabajar con ellos.
#Selección de las columnas que nos interesan, asignación de un nombre a dichas columnas, sustitución
#de las etiquetas de sentimiento por otras más descriptivas y eliminación de las celdas en las que el
#sentimiento del tweet ha sido etiquetado como "none".

def format_data(data):

  selected_data = data.iloc[:, [-1, -2]] # Seleccionar todas las filas; escoger penúltima y última columna
  selected_data.columns = ['polarity_value', 'tweet_text']
  # Cambiar N, P, NEU a negativo, positivo y neutral respectivamente
  labels = {'N': 'negative', 'P': 'positive', 'NEU': 'neutral', 'NONE': 'none'}
  selected_data['polarity_value'] = selected_data['polarity_value'].map(labels)

  # Eliminar filas donde el sentimiento sea "none"
  selected_data = selected_data[selected_data['polarity_value'] != 'none']

  # Retornar solo las dos columnas que nos interesan
  return selected_data[['tweet_text', 'polarity_value']]

In [92]:
import pandas as pd

#Cargamos dataset para comprobar los resultados
#Para ello, tenemos en cuenta que los datos estan estructurados en formato csv
data_url_test = "https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/refs/heads/master/1-Machine_Learning_Classic/Analisis_sentimientos_Twitter/spanish/datasets/Corpus/tass_2017/InterTASS/InterTASS_global.csv"
sentiment_data = pd.read_csv(data_url_test, header=None)
sentiment_data.head(10)

,0,1,2
0,770567971701940224,@LonelySoad mientras que no te pillen la prime...,N
1,770503386789711872,@ceemeese ya era hora de volver al csgo y deja...,P
2,770502863017635840,@mireiaescribano justo cuando se terminan las ...,P
3,770599972102348800,@LuisMartinez22_ pensba q iba a hacer @wxplosi...,N
4,770599962216390656,"@Vic_Phantomhive Si lo encuentro, sin compañer...",P
5,770599405816864768,"@MentalistGirl @PamplingNews lo sé, lo sé...la...",P
6,770453393714348032,Eso de echar de menos tu propia casa son leyen...,P
7,770466143098241025,"@kaveri_miguel yo estoy igual, he dormido nada...",N
8,770536258682617856,@Aymerich97 yo solo puse las evos de eevee,NONE
9,770613199200620545,@Caspitoo el día que te vi en persona estuvist...,N


In [93]:
#Preparamos los datos y aplicamos la transformación
#Imprimimos en pantalla las primeras diez instancias del dataset
data = format_data(sentiment_data)
data.head(10)

<ipython-input-91-800da3ff8acb>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['polarity_value'] = selected_data['polarity_value'].map(labels)


,tweet_text,polarity_value
0,@LonelySoad mientras que no te pillen la prime...,negative
1,@ceemeese ya era hora de volver al csgo y deja...,positive
2,@mireiaescribano justo cuando se terminan las ...,positive
3,@LuisMartinez22_ pensba q iba a hacer @wxplosi...,negative
4,"@Vic_Phantomhive Si lo encuentro, sin compañer...",positive
5,"@MentalistGirl @PamplingNews lo sé, lo sé...la...",positive
6,Eso de echar de menos tu propia casa son leyen...,positive
7,"@kaveri_miguel yo estoy igual, he dormido nada...",negative
9,@Caspitoo el día que te vi en persona estuvist...,negative
10,Echo de menos las sudaderas y los pantalones l...,negative


In [94]:
# Ejecutar las predicciones y obtener nuevas columna con dichas predicciones
data["sentiwordnet_sum_prediction"] = data["tweet_text"].apply(compute_sentiment_sum)
data["sentiwordnet_mean_prediction"] = data["tweet_text"].apply(compute_sentiment_mean)

# Mostrar 5 filas aleatorias del dataset
data.sample(5)

Se han truncado las últimas 5000 líneas del flujo de salida.
["SentiSynset('no.n')"]
Positive score for no: 0.0
Negative score for no: 0.75
Sentiment value: -0.75
 
None
@May_BloodArmor
n
@may_bloodarmor
['@SalvaSuay', 'No', 'te', 'veo', 'desde', '2013', ',', '¿', 'lo', 'ves', 'normal', '?']
[('@SalvaSuay', 'VERB'), ('No', 'ADV'), ('te', 'PRON'), ('veo', 'VERB'), ('desde', 'ADP'), ('2013', 'NOUN'), (',', 'PUNCT'), ('¿', 'PUNCT'), ('lo', 'PRON'), ('ves', 'VERB'), ('normal', 'ADJ'), ('?', 'PUNCT')]
@SalvaSuay
v
@salvasuay
No
r
no
["SentiSynset('no.n')"]
Positive score for no: 0.0
Negative score for no: 0.75
Sentiment value: -0.75
te
None
veo
v
ver
["SentiSynset('ver.v')"]
Positive score for ver: 0.125
Negative score for ver: 0.875
Sentiment value: -2.25
desde
None
2013
n
2013
,
None
¿
None
lo
None
ves
v
ver
["SentiSynset('ver.v')"]
Positive score for ver: 0.125
Negative score for ver: 0.875
Sentiment value: -4.5
normal
a
normal
?
None
['Porque', 'he', 'sido', 'muy', 'buena']
[('Porque', 

,tweet_text,polarity_value,sentiwordnet_sum_prediction,sentiwordnet_mean_prediction
506,@_Miss_Donovan Laura! Que luego te distraes d...,negative,neutral,positive
234,Que rápido pasa el tiempo solo se que cobro ca...,negative,negative,negative
1580,@bau_er no. Del rosa paso al negro mate y la s...,negative,negative,negative
735,@SanchezBurgos Suelo leer críticas muy diversa...,neutral,negative,negative
593,Han aparecido unas niñas jugando a las palmas ...,negative,negative,negative


In [95]:
from sklearn.metrics import classification_report

In [96]:
# Mostrar el informe de clasificación para cada uno de los métodos
print("Resultados del primer método de la suma:")
print(classification_report(data['polarity_value'], data['sentiwordnet_sum_prediction']))
print()
print("Resultados del segundo método de la media:")
print(classification_report(data['polarity_value'], data['sentiwordnet_mean_prediction']))

Resultados del primer método de la suma:
              precision    recall  f1-score   support

    negative       0.50      0.82      0.62       767
     neutral       0.13      0.17      0.15       216
    positive       0.64      0.07      0.12       642

    accuracy                           0.44      1625
   macro avg       0.42      0.35      0.30      1625
weighted avg       0.50      0.44      0.36      1625


Resultados del segundo método de la media:
              precision    recall  f1-score   support

    negative       0.49      0.79      0.61       767
     neutral       0.05      0.01      0.02       216
    positive       0.50      0.28      0.36       642

    accuracy                           0.48      1625
   macro avg       0.35      0.36      0.33      1625
weighted avg       0.43      0.48      0.43      1625



Vamos a añadir algunas reglas heurísticas que tengan en cuenta la negación, intensificadores y atenuadores.

In [113]:
# Lista de palabras de negación
negations = {"no", "nunca", "ninguno", "ninguna", "ningunos", "ningunas", "nada", "nadie"}

# Diccionario de modificadores de intensidad (incluye intensificadores y atenuadores)
intensity_modifiers = {
    # Intensificadores (factor > 1)
    "mucho": 1.5,
    "extremadamente": 1.8,
    "bastante": 1.2,
    "realmente": 1.3,
    "absolutamente": 1.8,
    "extremadamente": 1.4,
    "increíblemente": 1.5,
    # Atenuadores (factor < 1)
    "poco": 0.8,
    "apenas": 0.5,
    "difícilmente": 0.5,
    "escasamente": 0.7,
    "marginally": 0.6,
}

def compute_sentiment_sum(text):
    doc = nlp_es(text)
    tokenized_sent = [token.text for token in doc]

    pos_tagged_tokens = [(token.text, token.pos_) for token in doc]

    sentiment_value = 0.0

    for i, (word_text, tag) in enumerate(pos_tagged_tokens):
        wn_tag = map_spacy_tag(tag)
        if wn_tag is None:
            continue
        word = next((token for token in doc if token.text == word_text), None)
        if word is None:
          continue

        lemma = word.lemma_
        if not lemma:
          continue

        # Verificar si la palabra lematizada está en el diccionario de SentiWordNet
        # y crear la lista de synsets
        swn_synsets = []
        if lemma in spanish_sentiwordnet:
            # Obtener los puntajes positivos, negativos y los códigos de los synsets
            pos_tag, pos_score_lemma, _, neg_score_lemma, synset = spanish_sentiwordnet[lemma]

            for key, value in spanish_sentiwordnet.items():
              # Comprobar si el synset de la palabra coincide con el synset del diccionario
              if synset == value[4]:  # value[4] es el synset
                  swn_synsets.append(f"SentiSynset('{key}.{pos_tag}')")

            if len(swn_synsets) == 0:
              continue

            word_sentiment = pos_score_lemma - neg_score_lemma

            # Heurística de Negación: ventana de 5 tokens a la izquierda
            window_start = max(0, i - 5)
            window = pos_tagged_tokens[window_start:i]
            if any(tok[0].lower() in negations for tok in window):
                word_sentiment = -word_sentiment

            # Heurística de Modificadores de Intensidad/Atenuación:
            if i > 0:
                prev_word = pos_tagged_tokens[i-1][0].lower()
                if prev_word in intensity_modifiers:
                    word_sentiment *= intensity_modifiers[prev_word]

            sentiment_value += word_sentiment

        #If lemma not found, skip this word and move to next
        else:
            continue

    if sentiment_value >= 0.05:
        sentiment_label = "positive"
    elif sentiment_value <= -0.05:
        sentiment_label = "negative"
    else:
        sentiment_label = "neutral"

    return sentiment_label

In [114]:
def compute_sentiment_mean(text):
    doc = nlp_es(text)
    tokenized_sent = [token.text for token in doc]

    pos_tagged_tokens = [(token.text, token.pos_) for token in doc]

    pos_score_total = 0.0
    neg_score_total = 0.0
    obj_score_total = 0.0
    synset_count = 0

    for i, (word_text, tag) in enumerate(pos_tagged_tokens):
        wn_tag = map_spacy_tag(tag)
        if wn_tag is None:
            continue
        word = next((token for token in doc if token.text == word_text), None)
        if word is None:
          continue

        lemma = word.lemma_
        if not lemma:
          continue

        # Initialize pos_score_lemma, neg_score_lemma, and obj_score_lemma before the if statement
        pos_score_lemma = 0.0
        neg_score_lemma = 0.0
        obj_score_lemma = 0.0

        # Initialize pos_score, neg_score, and obj_score before the if statement
        pos_score = 0.0
        neg_score = 0.0
        obj_score = 0.0

        if lemma in spanish_sentiwordnet:
          # Obtener los puntajes positivos, negativos y los códigos de los synsets
          pos_tag, pos_score_lemma, obj_score_lemma, neg_score_lemma, synset = spanish_sentiwordnet[lemma]

          swn_synsets = []
          for key, value in spanish_sentiwordnet.items():
            # Comprobar si el synset de la palabra coincide con el synset del diccionario
            if synset == value[4]:  # value[4] es el synset
                swn_synsets.append(f"SentiSynset('{key}.{pos_tag}')")

          if len(swn_synsets) == 0:
            continue

        # Heurística de Negación: ventana de 5 tokens
        window_start = max(0, i - 5)
        window = pos_tagged_tokens[window_start:i]
        if any(tok[0].lower() in negations for tok in window):
            pos_score, neg_score = neg_score, pos_score  # Intercambia para reflejar la inversión

        # Heurística de Modificadores de Intensidad/Atenuación:
        if i > 0:
            prev_word = pos_tagged_tokens[i-1][0].lower()
            if prev_word in intensity_modifiers:
                multiplier = intensity_modifiers[prev_word]
                pos_score *= multiplier
                neg_score *= multiplier
                obj_score *= multiplier

        pos_score_total += pos_score
        neg_score_total += neg_score
        obj_score_total += obj_score
        synset_count += 1

    if synset_count == 0:
        return "neutral"

    pos_score_mean = pos_score_total / synset_count
    neg_score_mean = neg_score_total / synset_count
    obj_score_mean = obj_score_total / synset_count
    max_value = max(pos_score_mean, neg_score_mean, obj_score_mean)
    if max_value == pos_score_mean:
        sentiment_label = "positive"
    elif max_value == neg_score_mean:
        sentiment_label = "negative"
    else:
        sentiment_label = "neutral"

    return sentiment_label


In [115]:
# Ejemplo de uso:
text_example = "No me gusta nada esta película. Es buena, pero es realmente aburrida."
print("Sentiment (sum):", compute_sentiment_sum(text_example))
print("Sentiment (mean):", compute_sentiment_mean(text_example))

Sentiment (sum): negative
Sentiment (mean): positive


In [116]:
# Ejecutar las predicciones y obtener nuevas columna con dichas predicciones
data["sentiwordnet_sum_improved_prediction"] = data["tweet_text"].apply(compute_sentiment_sum)
data["sentiwordnet_mean_improved_prediction"] = data["tweet_text"].apply(compute_sentiment_mean)

# Mostrar 5 filas aleatorias del dataset
data.sample(5)

,tweet_text,polarity_value,sentiwordnet_sum_prediction,sentiwordnet_mean_prediction,sentiwordnet_sum_improved_prediction,sentiwordnet_mean_improved_prediction
993,Y yo!! pero no soy famoso Lugar muy recomenda...,positive,negative,negative,negative,positive
305,@AbadesaLPD todas las críticas son calificativ...,negative,negative,negative,positive,positive
354,Echo de menos las barras libres del Tángara.. ...,neutral,negative,neutral,negative,positive
20,"Buenos dias, vamos a hacer algunos recados y a...",positive,neutral,positive,neutral,positive
836,La gente de mi nuevo piso es bien buen pedo,positive,negative,negative,negative,positive


In [117]:
# Mostrar el informe de clasificación para cada uno de los métodos
print("Resultados del primer método de la suma:")
print(classification_report(data['polarity_value'], data['sentiwordnet_sum_improved_prediction']))
print()
print("Resultados del segundo método de la media:")
print(classification_report(data['polarity_value'], data['sentiwordnet_mean_improved_prediction']))

Resultados del primer método de la suma:
              precision    recall  f1-score   support

    negative       0.49      0.77      0.60       767
     neutral       0.13      0.19      0.15       216
    positive       0.50      0.08      0.14       642

    accuracy                           0.42      1625
   macro avg       0.37      0.35      0.30      1625
weighted avg       0.45      0.42      0.36      1625


Resultados del segundo método de la media:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       767
     neutral       0.00      0.00      0.00       216
    positive       0.39      1.00      0.57       642

    accuracy                           0.39      1625
   macro avg       0.13      0.33      0.19      1625
weighted avg       0.16      0.39      0.22      1625



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
